In [1]:
import pymongo
import xmltodict as xtd
from gzip import GzipFile
import pandas as pd

In [2]:
client = pymongo.MongoClient()
db = client.swatford

In [3]:
def handle_desc(_,d):
    descs.append(dict(d))        
    return True

In [4]:
descs = []
xtd.parse(GzipFile("../data/MeSH/desc2017.gz"),
          item_depth=2,
          item_callback=handle_desc,force_list=["AllowableQualifier","TreeNumber"])

In [5]:
descs[0].keys()

dict_keys(['DescriptorUI', 'DescriptorName', 'DateCreated', 'DateRevised', 'DateEstablished', 'AllowableQualifiersList', 'HistoryNote', 'OnlineNote', 'PublicMeSHNote', 'PreviousIndexingList', 'PharmacologicalActionList', 'TreeNumberList', 'ConceptList'])

In [16]:
def ancestors(tnl):
    ans = []
    for tn in tnl:
        a = None
        for x in tn.split(".")[:-1]:
            if a is None:
                a=x
            else:
                a+=f".{x}"
            ans.append(a)
    return ans

In [17]:
to_insert = []
for d in descs:
#     print([x["TreeNumber"] for x in d["TreeNumberList"]])
    e = {}
    e["uid"] = d["DescriptorUI"]
    e["name"] = d["DescriptorName"]["String"]
    if "AllowableQualifiersList" in d:
        e["q"] = [{"uid":x["QualifierReferredTo"]["QualifierUI"],
                  "name":x["QualifierReferredTo"]["QualifierName"]["String"],
                  "abb":x["Abbreviation"]} for x in d["AllowableQualifiersList"]["AllowableQualifier"]]
    if "TreeNumberList" in d:
        e["tn"] = d["TreeNumberList"]["TreeNumber"]
        e["ancestors"] = ancestors(d["TreeNumberList"]["TreeNumber"])
    to_insert.append(e)

In [18]:
len(to_insert)

28472

In [19]:
db.medline.descs.insert_many(to_insert)